In [13]:
# ==================
# Library
# ==================
import warnings
warnings.simplefilter('ignore')
import math
import pandas as pd
import numpy as np
import lightgbm as lgb
import time
import datetime
import gc
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold,GroupKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from contextlib import contextmanager
import logging
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import log_loss
import sys
import time
import feather
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import wandb
import pickle
%matplotlib inline
from sklearn.metrics import average_precision_score

sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage
pd.set_option('display.max_columns', 300)

In [14]:
# ==================
# Constant
# ==================
ex = "012"
TRAIN_PATH = "../input/train.csv"
TEST_PATH = "../input/test.csv"
USER_PATH = "../input/user_x_anime.csv"
SUB_PATH = "../input/sample_submission.csv"
SAVE_OOF_PATH = f"../output/exp/ex{ex}_oof.npy"
SAVE_TEST_SUB_PATH = f"../output/exp/ex{ex}_test_sub.csv"
LOGGER_PATH = f"../output/exp/ex_{ex}.txt"
wandb.init(project="dss2", name = f"ex{ex}")

wandb: Currently logged in as: takoihirokazu (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-08-20 07:13:34.618664: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-08-20 07:13:34.618730: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.



InvalidVersionSpec: Invalid version '2.3.0>=2.1.0': invalid character(s)



In [15]:

# ===============
# Settings
# ===============

SEED = 0
N_SPLITS = 5
SHUFFLE = True
LGBM_PARAMS = {'num_leaves': 32,
               'min_data_in_leaf': 64,
               'objective': 'regression',
               'max_depth': -1,
               'learning_rate': 0.05,
               "boosting": "gbdt",
               "bagging_freq": 1,
               "bagging_fraction": 0.8,
               "bagging_seed": SEED,
               "verbosity": -1,
              'reg_alpha': 0.1,
              'reg_lambda': 0.3,
              'colsample_bytree': 0.7,
              'metric':"rmse",
              'num_threads':32,
         }

LGBM_FIT_PARAMS = {
    'num_boost_round': 5000,
    'early_stopping_rounds': 200,
    'verbose_eval': 1000,
}

load_feature = ["../output/fe/fe001.feather",
                "../output/fe/fe002.feather",
                "../output/fe/fe003.feather",
                "../output/fe/fe004.feather",
                "../output/fe/fe005.feather",
                "../output/fe/fe006.feather",
                "../output/fe/fe007.feather",
                "../output/fe/fe009.feather",
                "../output/fe/fe011.feather",
                "../output/fe/fe012.feather",]

wandb.config.lgbm_params = LGBM_PARAMS
wandb.config.fit_params = LGBM_FIT_PARAMS
wandb.config.features = load_feature

In [16]:
# ====================
# Function
# ====================

def calc_loss(y_true, y_pred):
    return  np.sqrt(mean_squared_error(y_true, y_pred))


@contextmanager
def timer(name):
    t0 = time.time()
    yield 
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')

In [17]:
LOGGER = logging.getLogger()
FORMATTER = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
setup_logger(out_file=LOGGER_PATH)

2021-08-20 07:16:14,239 - INFO - logger set up


<RootLogger root (DEBUG)>

In [18]:
# ====================
# Main
# ====================
train_raw = pd.read_csv(TRAIN_PATH)
y = train_raw["Score"]
df = None
for i in load_feature:
    if df is not None:
        _df = pd.read_feather(i)
        df  = pd.concat([df, _df], axis=1)
    else:
        df = pd.read_feather(i)
train = df.iloc[:len(train_raw)]
test = df.iloc[len(train_raw):].reset_index(drop=True)

In [21]:
with timer("lightgbm"):
    kf = KFold(n_splits=N_SPLITS,random_state=SEED, shuffle=SHUFFLE)
    y_oof = np.empty([len(train),])
    y_test = []
    drop_cols = []
    features = list(train.columns)
    features = [i for i in features if i not in drop_cols]
    feature_importances = pd.DataFrame()
    categorical_features = []
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train,y)):
        print('Fold {}'.format(fold + 1))
        with timer(f"fold {fold}"):
            x_train, y_train = train.iloc[train_idx][features], y.iloc[train_idx]
            x_val, y_val =train.iloc[valid_idx][features], y.iloc[valid_idx]
            print("train:",len(x_train))

            y_pred_valid, y_pred_test, valid_loss, importances, best_iter,_ = train_lgbm(
                        x_train, y_train, x_val, y_val,test[features],
                        categorical_features=categorical_features,
                        feature_name=features,
                        fold_id=fold,
                        lgb_params=LGBM_PARAMS,
                        fit_params=LGBM_FIT_PARAMS,
                        loss_func=calc_loss,
                        calc_importances=True
                    )

            y_oof[valid_idx] = y_pred_valid
            score = calc_loss(y[valid_idx], y_pred_valid)
            wandb.log({f"Fold{fold}":score})
            LOGGER.info(f'Fold{fold}:CV={score}')
            y_test.append(y_pred_test)
            feature_importances = pd.concat([feature_importances, importances], axis=0, sort=False)

    score = calc_loss(y, y_oof)
    np.save(SAVE_OOF_PATH, y_oof)
    #LOGGER.info({CV:score)
    wandb.log({"CV":score})

Fold 1
train: 4000
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[465]	training's rmse: 0.110109	valid_1's rmse: 0.32802


2021-08-20 07:18:43,638 - INFO - Fold0:CV=0.3280197829250059
2021-08-20 07:18:43,648 - INFO - [fold 0] done in 8 s


Fold 2
train: 4000
Training until validation scores don't improve for 200 rounds
[1000]	training's rmse: 0.0448731	valid_1's rmse: 0.317338
[2000]	training's rmse: 0.0190415	valid_1's rmse: 0.316361
[3000]	training's rmse: 0.0123723	valid_1's rmse: 0.315962
[4000]	training's rmse: 0.00891377	valid_1's rmse: 0.315705
[5000]	training's rmse: 0.00652315	valid_1's rmse: 0.315539
Did not meet early stopping. Best iteration is:
[5000]	training's rmse: 0.00652315	valid_1's rmse: 0.315539


2021-08-20 07:19:38,714 - INFO - Fold1:CV=0.315539160551106
2021-08-20 07:19:38,722 - INFO - [fold 1] done in 55 s


Fold 3
train: 4000
Training until validation scores don't improve for 200 rounds
[1000]	training's rmse: 0.0518898	valid_1's rmse: 0.348367
Early stopping, best iteration is:
[887]	training's rmse: 0.0589136	valid_1's rmse: 0.348175


2021-08-20 07:19:55,987 - INFO - Fold2:CV=0.3481753490468285
2021-08-20 07:19:55,996 - INFO - [fold 2] done in 17 s


Fold 4
train: 4000
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[434]	training's rmse: 0.116085	valid_1's rmse: 0.336309


2021-08-20 07:20:07,311 - INFO - Fold3:CV=0.336309043482623
2021-08-20 07:20:07,331 - INFO - [fold 3] done in 11 s


Fold 5
train: 4000
Training until validation scores don't improve for 200 rounds
[1000]	training's rmse: 0.0455548	valid_1's rmse: 0.341801
[2000]	training's rmse: 0.0207942	valid_1's rmse: 0.341213
Early stopping, best iteration is:
[2212]	training's rmse: 0.0187835	valid_1's rmse: 0.341106


2021-08-20 07:20:49,780 - INFO - Fold4:CV=0.34110589514564704
2021-08-20 07:20:49,790 - INFO - [fold 4] done in 42 s
2021-08-20 07:20:49,813 - INFO - [lightgbm] done in 134 s


In [22]:
y_test_sub =  np.mean(y_test,axis=0)
sub = pd.read_csv(SUB_PATH)

In [23]:
sub["Score"] = y_test_sub
sub.to_csv(SAVE_TEST_SUB_PATH,index=False)

In [24]:
feature_importances.groupby(by="feature").mean().sort_values(by="gain",ascending=False).iloc[:20]

,gain,split,fold
feature,,,
Favorites,5033.984591,152.6,2.0
Favorites_Dropped_rate,2703.510034,430.8,2.0
On-Hold_Dropped_rate,2124.316841,803.6,2.0
Dropped_Plan to Watch_diff,1499.956745,238.2,2.0
On-Hold_Dropped_diff,1034.699904,539.4,2.0
anime_id_emb_16,850.268829,1226.6,2.0
Members_Watching_diff,607.344609,72.0,2.0
Members_Dropped_diff,593.665527,66.6,2.0
Members,488.156196,75.0,2.0
